<a href="https://colab.research.google.com/github/chhaewxn/huggingface_lab/blob/main/Hugging_Face_3%EC%A3%BC%EC%B0%A8_%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Transformers 실습하며 배우기 🤗

작성자: Hugging Face KREW, [김하림](https://github.com/harheem)

트랜스포머 기반의 사전 학습 모델의 뛰어난 점은 엄청난 양의 텍스트 데이터를 사용해 단어와 문장의 관계를 스스로 학습한다는 점이에요. 책, 뉴스, 웹 문서 등 다양한 텍스트를 분석하며 언어의 구조와 의미를 이해하게 된답니다.

하지만 이렇게 학습한 모델을 특정 작업에 활용하려면 추가 학습이 필요할 수 있어요. 이를 '파인튜닝'이라고 합니다. 파인튜닝은 적은 양의 labeled 데이터를 사용해 모델을 우리가 원하는 작업에 맞게 조정하는 과정을 의미해요.

저번 주에는 이러한 파인튜닝을 위한 데이터셋을 어떻게 준비하는지 배워보았습니다. 이번 주에는 진짜 파인튜닝을 해보죠! 🤗

### #문제 007

DistillBertForSequenceClassification 클래스를 사용해서 사전학습 모델을 불러와보세요. 그리고 코랩의 GPU 디바이스로 전달해보세요.

> from_pretrained를 사용해서 distilbert-base-uncased 모델을 불러옵니다. 불러온 모델의 신경망 구조를 확인하기 위해 model을 입력하고 코드를 실행해보세요.

> torch.cuda.is_available()이 True를 출력하면, GPU 이용이 가능합니다. 그리고 torch.tensors 타입의 데이터는 model.to(device) 메서드를 통해 GPU로 전달됩니다.

💡 HINT

이번에는 힌트가 없어요. 문제 속에 답이 있답니다 🤗

In [ ]:
# 라이브러리 import 하기
from transformers import DistilBertForSequenceClassification

# distilbert 모델 불러오기
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
import torch

# GPU 사용이 가능한 경우 텐서 타입 데이터를 GPU로 전달해보세요
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
device

device(type='cuda')

### #문제 008

TrainArguments 클래스를 호출해서, 파인튜닝에 사용하기 위한 하이퍼파라미터를 정의하세요.

> 딥러닝 신경망 모델에서 하이퍼파라미터는 모델 최적화 과정에 관여하는 조절 가능한 파라미터를 의미합니다. 하이퍼파라미터 값을 변경함으로 모델 학습과정에 영향을 미칩니다.

> 💡 데이터세트를 모델에 반복적으로 입력하는 횟수를 에포크라고 합니다. 한편, 모델의 신경망에 주입되어 파라미터를 갱신하는 데이터 샘플 그룹의 크기를 배치라고 합니다. 딥러닝 모델의 확률적 경사 하강법(SGD)에서 사용되는 배치는 특별히 미니 배치로 불립니다. 따라서 미니 배치는 배치의 한 유형이라고 볼 수 있습니다. 학습률은 각 (미니) 배치 또는 에포크마다 모델 파라미터를 얼마나 조정할지 결정하는 비율입니다. 이 학습률이 너무 낮으면 학습 과정이 지나치게 느려지고, 반대로 너무 높으면 학습 과정이 불안정해질 수 있습니다.

💡 HINT

TrainArguments를 호출해보세요. 대표적인 하이퍼파라미터들의 값을 다음과 같이 설정합니다.

(미니 배치 사이즈 16의 의미는 각기 16회의 추론 및 오차 계산이 실행된다는 의미입니다.)
- 학습 에포크: 8
- 학습 시 미니 배치 사이즈: 16
- 평가 시 미니 배치 사이즈: 64
- 최적화 함수: Adam

> 못 풀겠어도 걱정하지 마세요. [이 문서](https://huggingface.co/docs/transformers/v4.21.2/en/main_classes/trainer#transformers.TrainingArguments)에서 답을 찾을 수 있어요 👀

> 잘 정리된 [블로그 글](https://velog.io/@nkw011/Tutorial4-Fine-tune-a-pretrained-model)도 추천드려요 ✨

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',           # 출력 디렉토리 경로
    num_train_epochs=8,               # 학습(training) 에포크 수
    per_device_train_batch_size=16,   # 학습 시 (디바이스별) 미니 배치 수
    per_device_eval_batch_size=64,    # 평가 시 (디바이스별) 미니 배치 수
    warmup_steps=500,                 # 학습률 스케줄링용 웜업 스텝 수
    weight_decay=0.01,                # 가중치 감소 감도
    logging_dir='./logs',             # 로그 디렉토리 경로
    logging_steps=10,

)

### #문제 009

문제 004부터 008까지의 과정에 기반한 Trainer 클래스를 인스턴스화하고 파인튜닝해보세요. 키워드 인수로 model, args, train_dataset, eval_dataset을 명확하게 전달하세요. 그리고 파인튜닝 전후에 다음 세 문장 각각의 감정 판단 결과를 비교해보세요.

- "I feel fantastic" 나는 환상적인 기분이에요.
- "My life is going something wrong." 내 인생이 무언가 잘못돼 가고 있어.
- "I have not figured out what the chosen title has to do with the movie." 선정된 영화 제목이 영화 내용과 무슨 관계가 있는지 모르겠어요.

💡 HINT

파인튜닝 절차는 Trainer.train()을 사용하면 매우 쉽게 수행할 수 있습니다.

In [ ]:
# distilbert-base-uncased 모델에서 토크나이저 불러오기
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# 파인튜닝 이전에 세 입력 문장 극성 판별
input_tokens = tokenizer(["I feel fantastic", "My life is going something wrong.", "I have not figured out what the chosen title has to do with the movie."], truncation=True, padding=True)

# 입력 문장 토크나이징 결과(input_tokens)에 담긴 input_ids를 모델에 투입
# 그리고 모델 출력 결과를 GPU로 전송하며 값은 변수 outputs에 저장
outputs = model(torch.tensor(input_tokens['input_ids']).to(device))

# 레이블 딕셔너리 생성
label_dict = {1:'positive', 0:'negative'}

# outputs 변수에 담긴 logits 값을 행 단위, 즉 입력 문장 단위로 가장 큰 값 위치(인덱스) 추출
# 그 결과값(인덱스)을 cpu로 넘기고 넘파이 타입으로 변경 후, 인덱스에 매칭되는 레이블 출력
print([label_dict[i] for i in torch.argmax(outputs['logits'], axis=1).cpu().numpy()])

['negative', 'negative', 'negative']


In [ ]:
!pip -q install pyarrow==15.0.2 datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# 데이터셋 로드
dataset = load_dataset('stanfordnlp/imdb')

# 데이터셋 샘플링 및 분할
train_dataset = dataset['train'].shuffle(seed=923).select(range(1000))
test_dataset = dataset['test'].shuffle(seed=923).select(range(1000))
split_dataset = train_dataset.train_test_split(test_size=0.2, seed=923)

train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

# 토큰화 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

# 데이터셋에 토큰화 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 필요한 열만 선택하고 형식 변경
train_dataset = train_dataset.remove_columns(['text']).rename_column("label", "labels")
valid_dataset = valid_dataset.remove_columns(['text']).rename_column("label", "labels")
test_dataset = test_dataset.remove_columns(['text']).rename_column("label", "labels")

# 데이터 형식을 pytorch로 설정
train_dataset.set_format('torch')
valid_dataset.set_format('torch')
test_dataset.set_format('torch')

# 결과 확인
print("학습 데이터 크기:", len(train_dataset))
print("검증 데이터 크기:", len(valid_dataset))
print("테스트 데이터 크기:", len(test_dataset))

# 데이터 샘플 확인
print("\n학습 데이터 샘플:")
sample = train_dataset[0]
print("Input IDs:", sample['input_ids'][:10], "…")  # 처음 10개 토큰만 출력
print("Attention Mask:", sample['attention_mask'][:10], "…")
print("Label:", sample['labels'])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

학습 데이터 크기: 800
검증 데이터 크기: 200
테스트 데이터 크기: 1000

학습 데이터 샘플:
Input IDs: tensor([  101,  2798, 22953, 15551,  3340,  2004,  8318, 25657,  1037,  2610]) …
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) …
Label: tensor(0)


In [ ]:
# 런타임 약 4 혹은 5분 소요됩니다
# 딥러닝 모델의 특성상 결과가 매번 약간 다르게 나올 수 있어요
from transformers import Trainer

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=valid_dataset
)

trainer.train()

Step,Training Loss
10,0.686800
20,0.678500
30,0.676600
40,0.675500
50,0.667700
60,0.644800
70,0.672600
80,0.614300
90,0.530600
100,0.484400


TrainOutput(global_step=400, training_loss=0.25634606504114343, metrics={'train_runtime': 297.5648, 'train_samples_per_second': 21.508, 'train_steps_per_second': 1.344, 'total_flos': 847791351398400.0, 'train_loss': 0.25634606504114343, 'epoch': 8.0})

In [ ]:
# Trainer.train()에 의한 파인튜팅 이후 세 입력 문장 극성 판별
input_tokens = tokenizer(["I feel fantastic", "My life is going something wrong.", "I have not figured out what the chosen title has to do with the movie."], truncation=True, padding=True)
outputs = model(torch.tensor(input_tokens['input_ids']).to(device))


label_dict = {1:'positive', 0:'negative'}

print([label_dict[i] for i in torch.argmax(outputs['logits'], axis=1).cpu().numpy()])

['positive', 'negative', 'negative']


어떠셨나요? 🤗 Transformers 라이브러리를 이용하여 아주 간단하게 파인튜닝을 수행할 수 있었고, 개선된 결과도 확인할 수 있으셨나요? 현재 예제는 영어로 되어 있지만, 다음 Masters 기간 동안에는 한국어로 파인튜닝할 수 있는 예제를 만들어 보는 것은 어떨까요?

이제 Challengers 기간이 거의 종료되었습니다. Masters 기간에는 더욱 더 멋진 프로젝트와 활동이 기다리고 있으니, 함께 끝까지 달려주세요! 🏃🏻‍♂️‍➡️🇰🇷